# Motion GPS Module

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [sse-results](https://github.com/Logiqx/sse-results) and is distributed under the terms of the GNU General Public License.

sse-results is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

sse-results is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with sse-results. If not, see <https://www.gnu.org/licenses/>.

In [1]:
import os
import logging

import json

import socket
from urllib.request import urlopen

import unittest

from common import projdir, testExit

from constants import *

## Default Values

Default settings, just in case of missing details in the config file

In [2]:
# Default logging level
DEFAULT_LOGGING_LEVEL = logging.INFO

In [3]:
# Communications defaults
DEFAULT_HTTP_PORT=80
DEFAULT_PING_TIMEOUT=5
DEFAULT_JSON_TIMEOUT=10

In [4]:
# Motion status
HOST_STATUS_UNKNOWN = 0

HOST_STATUS_CONNECTED = 1
HOST_STATUS_INSPECTED = 2
HOST_STATUS_COMPLETED = 3

HOST_STATUS_REFUSED = -1
HOST_STATUS_TIMEOUT = -2
HOST_STATUS_GAIERROR = -3

In [5]:
# Default threshold for Motion filesystem warning
DEFAULT_FILESYSTEM_THRESHOLD = 95

# Default threshold for Motion battery warning
DEFAULT_BATTERY_THRESHOLD = 60

# Default GNSS rate
DEFAULT_GNSS_RATE = 5

# Default one log per day
DEFAULT_ONE_LOG = 0

# Default timezone (UTC)
DEFAULT_TIMEZONE = 0

# Default LED flash
DEFAULT_FLASH = 1

## Motion Class

Stuff relating to a single Motion device

In [6]:
class Motion():
    '''Motion GPS'''
    
    def __init__(self, config, identifier, fileId=None):
        """Simple init method"""

        self.config = config       
        self.identifier = int(identifier)
        self.fileId = fileId

        self.hostname = f'motion-{identifier}'

        self.addrInfo = None
        self.status = HOST_STATUS_UNKNOWN
        
        self.users = {}

        
    def getAddrInfo(self, port=DEFAULT_HTTP_PORT):
        """Get address information (IP address) for Motion"""

        try:
            addrInfo = socket.getaddrinfo(self.hostname, port, proto=socket.IPPROTO_TCP)[0][4]

            # Sky SR101 router will sometimes return 10.10.10.10 and should be treated the same as socket.gaierror
            if addrInfo[0] == '10.10.10.10':
                raise socket.gaierror

        # Only really expect a socket.gaierror exception, either from socket.getaddrinfo or because of 10.10.10.10
        except socket.gaierror:
            self.addrInfo = None

        # Just in case of some other (unexpected) issue!
        except Exception:
            self.addrInfo = None

        # The IP address of the Motion has been successfully identified. Yay!
        else:
            self.addrInfo = addrInfo
            
        return self.addrInfo


    def getStatus(self, timeout=DEFAULT_PING_TIMEOUT):
        """Check the status of the Motion

        Note the use of self.addrInfo instead of using the hostname with s.connect((host, port)).

        This avoids erroneous 'gaierror: [Errno -5] No address associated with hostname' under certain circumstances."""

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()
        
        # Status can only be checked if address information is available
        if self.addrInfo:
            try:
                # Attempt connection to the standard HTTP socket as determined by socket.getaddrinfo()
                s = socket.socket()
                s.settimeout(timeout)
                s.connect(self.addrInfo)
                
                # Close the connection in a timely fashion - good practice
                s.shutdown(socket.SHUT_RDWR)
                s.close()
                
            # This should only happen if the IP address does not belong to a Motion
            except ConnectionRefusedError:
                self.status = HOST_STATUS_REFUSED

            # This should only happen when the Motion has been disconnected from the WiFi
            except socket.timeout:
                self.status = HOST_STATUS_TIMEOUT

            # Just in case of some other (unexpected) issue!
            except Exception:
                self.status = HOST_STATUS_UNKNOWN

            # The Motion is accepting connections. Yay!
            else:
                self.status = HOST_STATUS_CONNECTED

        # Address information is not known
        else:
            self.status = HOST_STATUS_GAIERROR

        return self.status
   

    def fetchJson(self, path, port=DEFAULT_HTTP_PORT, timeout=DEFAULT_JSON_TIMEOUT):
        """Fetch JSON from the Motion"""

        result = None

        # Ensure address information is available
        if self.addrInfo is None:
            self.getAddrInfo()

        # JSON can only be fetched if address information is available
        if self.addrInfo:
            # Use addrInfo to avoid unnecessary DNS lookup which can take several seconds
            url = 'http://{}/{}'.format(self.addrInfo[0], path)

            try:
                # The urlopen() function is a simple but effective way to open a URL
                response = urlopen(url, timeout=timeout)
                txt = response.read()

            except Exception as e:
                logging.error("%s was unable to provide %s", self.hostname, path)
                raise

            # Parse the JSON and store it in a dictionary
            result = json.loads(txt)

        return result


    def fetchInfo(self):
        """Fetch info.json from the Motion"""

        self.info = self.fetchJson('info.json')


    def fetchSettings(self):
        """Fetch settings.json from the Motion"""

        self.settings = self.fetchJson('settings.json')


    def fetchLogs(self):
        """Fetch logs.json from the Motion"""

        self.logs = self.fetchJson('logs.json')


    def checkInfo(self):
        """Check Motion information such as firmware, storage and battery"""
        
        self.checkFirmware()
        self.checkStorage()
        self.checkBattery()


    def checkFirmware(self):
        """Check firmware is acceptable"""
        
        try:
            # 3144 onwards introduced code, name and special
            firmware = self.info['motion']['firmware']['code']
        except Exception:
            # Prior to 3144 it was just the code
            firmware = self.info['motion']['firmware']
        
        # Only specific firmwares are acceptable
        try: 
            firmwares = self.config['Motion']['Info']['Firmwares']
        except Exception:
            firmwares = {}

        if str(firmware) not in firmwares:
            print(firmwares)
            logging.warning("%s firmware is not approved (%s)", self.hostname, firmware)
        else:
            logging.debug("%s firmware is approved (%s)", self.hostname, firmware)


    def checkStorage(self):
        """Check storage is not above acceptable threshold"""
        
        try:
            level = self.info['motion']['filesystem']['level']
        except Exception:
            logging.warning("%s filesystem level (% storage used) was undetermined", self.hostname)
            level = 0        
            
        try: 
            threshold = self.config['Motion']['Info']['Thresholds']['Storage']
        except Exception:
            threshold = DEFAULT_FILESYSTEM_THRESHOLD

        if int(level) > threshold:
            logging.warning("%s storage is %s%% full", self.hostname, level)
        else:
            logging.debug("%s storage is %s%% full", self.hostname, level)


    def checkBattery(self):
        """Check battery is not below acceptable threshold"""
        
        try:
            level = self.info['motion']['battery']['level']
        except Exception:
            logging.warning("%s battery level undetermined", self.hostname)
            level = 0        
            
        try: 
            threshold = self.config['Motion']['Info']['Thresholds']['Battery']
        except Exception:
            threshold = DEFAULT_BATTERY_THRESHOLD

        if int(level) < threshold:
            logging.warning("%s battery level is %s%%", self.hostname, level)
        else:
            logging.info("%s battery level is %s%%", self.hostname, level)


    def checkSettings(self):
        """Check Motion settings such as firmware, filesystem and battery"""

        self.checkGnssRate()
        self.checkOneLog()
        self.checkTimezone()
        self.checkFlash()
    

    def checkGnssRate(self):
        """Check GNSS rate is correct"""
        
        self.checkSetting('GNSS rate', 'GNSS Rate', 'gnss_rate', DEFAULT_GNSS_RATE)


    def checkOneLog(self):
        """Check logs per day is correct"""
        
        self.checkSetting('one log per day', 'One Log Per Day', 'one_log_per_day', DEFAULT_ONE_LOG)


    def checkTimezone(self):
        """Check timezone is correct"""
        
        self.checkSetting('timezone', 'Timezone', 'timezone', DEFAULT_TIMEZONE)


    def checkFlash(self):
        """Check LED flash is correct"""
        
        try: 
            actual = self.settings['settings']['custom_led']['flash']
            
            self.checkSetting('LED flash', 'LED Flash', 'flash', DEFAULT_FLASH, actual=actual)

        except Exception:
            logging.warning("%s LED Flash undetermined", self.hostname)       


    def checkSetting(self, description, configSetting, motionSetting, default, actual=None):
        """Check battery is not below acceptable threshold"""
        
        try:
            if actual is None:
                actual = self.settings['settings'][motionSetting]
           
            try: 
                expected = self.config['Motion']['Settings'][configSetting]
            except Exception:
                expected = default

            if int(actual) != expected:
                logging.warning("%s %s is %s", self.hostname, description, actual)
            else:
                logging.debug("%s %s is %s", self.hostname, description, actual)

        except Exception:
            logging.warning("%s %s undetermined", self.hostname, description)

## Unit Tests

Some basic unit tests, not relying upon Motion being connected to the WiFi

In [7]:
class TestMotions(unittest.TestCase):
    '''Class to test motions'''

    def testIdentifier(self):
        """Test identifier is as expected"""
        
        motion = Motion(config, 470, 'GEO21MIC')

        self.assertEqual(motion.identifier, 470)
        

    def testHostname(self):
        """Test hostname is as expected"""
        
        motion = Motion(config, 470, 'GEO21MIC')

        self.assertEqual(motion.hostname, 'motion-470')
        

    def testFileId(self):
        """Test file ID is as expected"""
        
        motion = Motion(config, 470, 'GEO21MIC')

        self.assertEqual(motion.fileId, 'GEO21MIC')

## Run Unit Tests

Note: Only run unit tests when running this script directly, not during an import

In [8]:
def loadConfig():
    '''Read server config from JSON'''

    filename = os.path.join(projdir, CONFIG_DIR, 'server.json')
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        config = json.loads(jsonTxt)

    return config

In [9]:
def initLogging(config):
    """Initialise logging"""
    
    formatString = "%(asctime)s,%(levelname)s,%(message)s"
    dateTimeFormat = "%Y-%m-%d %H:%M:%S"

    try:
        level = config['Logging']['Level']
    except KeyError:
        level = DEFAULT_LOGGING_LEVEL

    logging.basicConfig(format=formatString, datefmt=dateTimeFormat, level=level)

In [10]:
if __name__ == '__main__':
    
    config = loadConfig()
    initLogging(config)
    
    unittest.main(argv=['first-arg-is-ignored'], exit=testExit)

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK
